In [14]:
import pandas as pd
from datetime import datetime, timedelta
import os

In [15]:
#Processes a single wide-format MISO Day-Ahead EPNode CSV and returns a long-format DataFrame with datetime, node, and LMP components.
#Data Source: https://www.misoenergy.org/markets-and-operations/real-time--market-data/market-reports/#nt=%2FMarketReportType%3AHistorical%20LMP%2FMarketReportName%3ADay-Ahead%20EPNode%20LMP%20(zip)&t=10&p=3&s=MarketReportPublished&sd=desc
def process_wide_format_safely(file_path):

    # Step 1: Read lines and find header row
    with open(file_path, 'r', encoding='utf-8') as f:
        lines = f.readlines()

    # Locate header row (should contain "EPNode" and "HE1")
    header_row_index = next(
        (i for i, line in enumerate(lines) if "EPNode" in line and "HE1" in line),
        None
    )

    if header_row_index is None:
        raise ValueError("Could not find valid header row in file.")

    # Step 2: Extract market date (e.g., "Market Day: 06/05/2025")
    market_day_line = next(
        (line for line in lines if "Market Day" in line),
        None
    )
    if not market_day_line:
        raise ValueError("Market Day not found in file.")

    market_day_str = market_day_line.split(":")[-1].strip()
    market_date = datetime.strptime(market_day_str, "%m/%d/%Y")

    # Step 3: Load CSV from the correct header row
    df = pd.read_csv(file_path, skiprows=header_row_index)

    # Step 4: Convert wide to long format
    long_data = []

    for i in range(0, len(df) - 2, 3):  # Process 3 rows at a time (LMP, MCC, MLC)
        try:
            row_lmp = df.iloc[i]
            row_mcc = df.iloc[i + 1]
            row_mlc = df.iloc[i + 2]

            # Safe access to first column (node name), drop leading "L"
            node = " ".join(str(row_lmp.iloc[0]).split()[1:])

            for hour in range(1, 25):  # HE1 to HE24
                hour_col = f"HE{hour}"
                dt = market_date + timedelta(hours=hour - 1)

                long_data.append({
                    "datetime": dt,
                    "node": node,
                    "lmp_total": float(row_lmp[hour_col]),
                    "congestion": float(row_mcc[hour_col]),
                    "loss": float(row_mlc[hour_col])
                })

        except Exception as e:
            print(f"Error processing rows {i}-{i+2}: {e}")
            continue

    return pd.DataFrame(long_data)

In [16]:
dirty_folder = "data/dirty/DA/Wide"
cleaned_folder = "data/dirty/DA/Long"

os.makedirs(cleaned_folder, exist_ok=True)  # make sure cleaned folder exists
file_path = 'data/cleaned/DA/miso_da_combined_clean.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    for filename in sorted(os.listdir(dirty_folder)):
        if filename.startswith("DA_Load_EPNodes") and filename.endswith(".csv"):
            input_path = os.path.join(dirty_folder, filename)
            output_path = os.path.join(cleaned_folder, filename.replace("DA_Load_EPNodes", "miso_da")[:-4] + "_clean.csv")
            
            print(f"Processing {filename} ...")
            try:
                df = process_wide_format_safely(input_path)
                df.to_csv(output_path, index=False)
                print(f"Saved cleaned file to {output_path}")
            except Exception as e:
                print(f"Failed to process {filename}: {e}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")


The file data/cleaned/DA/miso_da_combined_clean.csv does not exist. Running the processing code...
Processing DA_Load_EPNodes_20250401.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250401_clean.csv
Processing DA_Load_EPNodes_20250402.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250402_clean.csv
Processing DA_Load_EPNodes_20250403.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250403_clean.csv
Processing DA_Load_EPNodes_20250404.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250404_clean.csv
Processing DA_Load_EPNodes_20250405.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250405_clean.csv
Processing DA_Load_EPNodes_20250406.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250406_clean.csv
Processing DA_Load_EPNodes_20250407.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250407_clean.csv
Processing DA_Load_EPNodes_20250408.csv ...
Saved cleaned file to data/dirty/DA/Long/miso_da_20250408_clean.csv
Proce

In [17]:
def combine_cleaned_csvs(folder="data/dirty/DA/Long", prefix="miso_da_", ext=".csv"):
    all_dfs = []
    
    for filename in sorted(os.listdir(folder)):
        if filename.startswith(prefix) and filename.endswith(ext):
            file_path = os.path.join(folder, filename)
            print(f"📄 Loading: {filename}")
            try:
                df = pd.read_csv(file_path, parse_dates=["datetime"])
                all_dfs.append(df)
            except Exception as e:
                print(f"❌ Failed to load {filename}: {e}")

    if not all_dfs:
        print("❌ No files combined.")
        return pd.DataFrame()

    combined_df = pd.concat(all_dfs, ignore_index=True)
    combined_df.sort_values(by=["datetime", "node"], inplace=True)
    combined_df.to_csv("data/cleaned/DA/miso_da_combined_clean.csv")
    print(f"✅ Combined dataframe saved")
    return 

In [18]:
file_path = 'data/cleaned/DA/miso_da_combined_clean.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    combine_cleaned_csvs()
    # Make sure the directory exists before saving
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    print(f"Processing complete. File saved to {file_path}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")

The file data/cleaned/DA/miso_da_combined_clean.csv does not exist. Running the processing code...
📄 Loading: miso_da_20250401_clean.csv
📄 Loading: miso_da_20250402_clean.csv
📄 Loading: miso_da_20250403_clean.csv
📄 Loading: miso_da_20250404_clean.csv
📄 Loading: miso_da_20250405_clean.csv
📄 Loading: miso_da_20250406_clean.csv
📄 Loading: miso_da_20250407_clean.csv
📄 Loading: miso_da_20250408_clean.csv
📄 Loading: miso_da_20250409_clean.csv
📄 Loading: miso_da_20250410_clean.csv
📄 Loading: miso_da_20250411_clean.csv
📄 Loading: miso_da_20250412_clean.csv
📄 Loading: miso_da_20250413_clean.csv
📄 Loading: miso_da_20250414_clean.csv
📄 Loading: miso_da_20250415_clean.csv
📄 Loading: miso_da_20250416_clean.csv
📄 Loading: miso_da_20250417_clean.csv
📄 Loading: miso_da_20250418_clean.csv
📄 Loading: miso_da_20250419_clean.csv
📄 Loading: miso_da_20250420_clean.csv
📄 Loading: miso_da_20250421_clean.csv
📄 Loading: miso_da_20250422_clean.csv
📄 Loading: miso_da_20250423_clean.csv
📄 Loading: miso_da_20250424

In [19]:
# from https://www.eia.gov/electricity/wholesalemarkets/data.php?rto=miso
Load_Temp = pd.read_csv('data/dirty/Features/miso_load-temp_hr_2025.csv', skiprows = 3)
Wind = pd.read_csv('data/dirty/Features/miso_gen_wnd_hr_2025.csv', skiprows = 3)

In [20]:
Load_Temp = Load_Temp[['Local Timestamp Eastern Standard Time (Interval Beginning)', 'MISO Total Forecast Load (MW)', 'Indianapolis Temperature (Fahrenheit)']]
Wind = Wind[['Local Timestamp Eastern Standard Time (Interval Beginning)', 'MISO Total Wind Generation (MW)']]

In [21]:
# Basic merge on a single column
Load_Temp_Wind = pd.merge(Load_Temp, Wind, on='Local Timestamp Eastern Standard Time (Interval Beginning)')

# convert DataFrame date column to datetime format
Load_Temp_Wind['Local Timestamp Eastern Standard Time (Interval Beginning)'] = pd.to_datetime(Load_Temp_Wind['Local Timestamp Eastern Standard Time (Interval Beginning)'])

# Filter for April and May (months 4 and 5)
Load_Temp_Wind = Load_Temp_Wind[Load_Temp_Wind['Local Timestamp Eastern Standard Time (Interval Beginning)'].dt.month.isin([4, 5])]

#Rename columns
Load_Temp_Wind.columns = ['datetime', 'Total Forecast Load', 'Temperature', 'Total Wind Gen']

In [22]:
file_path = 'data/cleaned/Features/miso_load-temp-wind_hr_2025.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    Load_Temp_Wind.to_csv('data/cleaned/Features/miso_load-temp-wind_hr_2025.csv')
    print(f"Processing complete. File saved to {file_path}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")

The file data/cleaned/Features/miso_load-temp-wind_hr_2025.csv already exists. Skipping processing.


In [23]:
folder_path = 'data/dirty/DA_Constraints'  # Where your Excel files are
file_paths = glob.glob(os.path.join(folder_path, '*.xls'))

all_dfs = []

for path in file_paths:
    filename = os.path.basename(path)
    date_str = filename[:8]  # e.g., '20250401'
    file_date = pd.to_datetime(date_str, format='%Y%m%d')

    try:
        # Skip the first 3 rows
        df = pd.read_excel(path, skiprows=3)

        # Make sure "Hour of Occurrence" column exists
        if 'Hour of Occurrence' not in df.columns:
            raise ValueError(f"'Hour of Occurrence' not found in {filename}")

        # Create datetime column
        df['datetime'] = df['Hour of Occurrence'].astype(int).apply(
            lambda h: file_date + pd.Timedelta(hours=h)
        )

        all_dfs.append(df)

    except Exception as e:
        print(f"Failed to process {filename}: {e}")

# Combine all files into one DataFrame
combined_df = pd.concat(all_dfs, ignore_index=True)

# Convert string to datetime if needed
combined_df['datetime'] = pd.to_datetime(combined_df['datetime'])

# Sort by datetime column (ascending order - oldest to newest)
combined_df = combined_df.sort_values(by='datetime', ascending=False)

In [24]:
combined_df = combined_df[['Constraint Name', 'datetime', 'Shadow Price']]

# Suppose your constraint DataFrame is called constraints_df
constraint_pivot = combined_df.pivot_table(
    index='datetime',
    columns='Constraint Name',
    values='Shadow Price'
)

In [25]:
file_path = 'data/cleaned/DA_Constraints/combined_DA_constraints.csv'

if not os.path.exists(file_path):
    print(f"The file {file_path} does not exist. Running the processing code...")
    constraint_pivot.to_csv('data/cleaned/DA_Constraints/combined_DA_constraints.csv')    # Make sure the directory exists before saving
    os.makedirs(os.path.dirname(file_path), exist_ok=True)
    print(f"Processing complete. File saved to {file_path}")
else:
    print(f"The file {file_path} already exists. Skipping processing.")
    

The file data/cleaned/DA_Constraints/combined_DA_constraints.csv already exists. Skipping processing.
